In [35]:
%%capture
import sys
sys.path.append('/work/pikachu/utils/')
sys.path.append('/work/pikachu/third/')
sys.path.append('/work/pikachu/projects/kaggle/aslfr/third/NeMo')
sys.path.append('..')
from gezi.common import *
import gezi
from src.config import *
from src.vis import *
gezi.init_flags()

In [36]:
STEP = 3
FOLDS = 4
def gen_metrics(root, folds=4, step=STEP, force_full=False):
  dfs= []
  fold_counts = defaultdict(int)
  for fold in range(folds):
    pattern = f'{root}/{fold}/*/metrics.csv'
    files = glob.glob(pattern)
    if not files:
      continue
    files = sorted(files, key=lambda x: os.path.getmtime(x))
    for file in files:
      if not gezi.non_empty(file):
        continue
      df = pd.read_csv(file)
      if not len(df):
        continue
      if force_full and len(df) != step:
        continue
      if 'step' not in df.columns:
        df['step'] = range(len(df))
      model = os.path.basename(os.path.dirname(file))
      try:
        if df.step.max() >= step:
          fold_counts[model] += 1
      except Exception:
        # ic(df)
        continue
      df['model'] = model
      df['mtime'] = os.path.getmtime(file)
      df['ctime'] = os.path.getctime(file)
      df['fold'] = fold
      dfs.append(df)
      # dfs = sorted(dfs, key=lambda x: -len(x.columns))
  
  df = pd.concat(list(dfs))
  df['num_folds'] = df.model.apply(lambda x: fold_counts[x])
  return df

In [37]:
def show(metric, silent=False, num=100, folds=FOLDS):
  from gezi.plot import line
  d = df
  # d = d[d.step > 20]
  fig = line(d, metric, x='step', color='model', return_figs=True)[0]
  if not silent:
    print(f'{metric}: per fold')
    display(fig)

  # start = 0 if not silent else folds - 1
  # for fold in range(start, folds):
  #   print(f'{metric}: num_folds', fold + 1)
  #   d_ = d
  #   d = d[d.num_folds >= fold + 1]
  #   d = d[d.fold <= fold]
  #   if not len(d):
  #     continue
  #   d = d.groupby(['model'])[metric] \
  #     .aggregate(np.mean).reset_index()
  #   d = d[['model', metric]].sort_values([metric], ascending=False).reset_index(drop=True)
  #   d = d.head(num)
  #   d[f'{metric}_str'] = ['%.4f' % x for x in d[metric].values]
  #   display(px.bar(d, x=metric, y='model',  color='model', text=f'{metric}_str'))
  #   d = d_

In [38]:
v = '31'
# type = 'tiny'
# type = 'mid'
# type = 'large'
mark = 'offline'
# root = f'../working/{mark}/{v}.*'
root = f'../working/{mark}/{v}'

In [39]:
try:
  df = gen_metrics(root)
except Exception:
  df = gen_metrics(root, force_full=True)

In [40]:
df = df[df.model.str.contains('ep-400')]
df = df[df.model.str.contains('fat-14')]

In [41]:
df = df[~df.model.str.contains('epoch')]
df = df[~df.model.str.contains('bs-')]

In [42]:
show('score')

score: per fold
